In [1]:
import networkx as nx
import random as rnd
# import pdb

Read graph in kreach format

In [2]:
def read_kreach(filename):
    G = nx.DiGraph()
    with open(filename) as fin:
        n = 0
        m = 0
        for i, line in enumerate(fin):
            data = [int(s) for s in line.split()]
            if (i == 0):
                n = data[0]
                m = data[1]
            else:
                G.add_node(data[0])
                if (data[1] > 0):
                    for v in data[2:]:
                        G.add_edge(data[0], v)
    return G

In [27]:
G = read_kreach('agrocyc_dag_uniq.kreach')
k = 3
b = 1000

In [28]:
print("n: {}, m: {}".format(G.number_of_nodes(), G.number_of_edges()))

n: 12684, m: 13408


Construction of $D_1$ at the first level

In [5]:
def largest_degree_node(G):
    degrees = G.degree(G.nodes())
    return max(degrees.keys(), key=lambda key: degrees[key])

In [6]:
def forward_bfs(G, v, D, S, V, s=None, d=None):
    predecessors = {v: set()}
    dist = {v: 0}
    if (s is None):
        s = v
    else:
        predecessors[s] = set([v])
        dist[s] = d
    queue = [s]
    while (queue):
        u = queue.pop(0)
        if (u in S):
            D.add_edge(v, u, weight=dist[u])
        elif (not ((predecessors[u] - set([v])) & S)):
            V.add(u)
            D.add_edge(v, u, weight=dist[u])
        if (dist[u] >= k):
            continue
        for nxt in G.successors(u):
            if (nxt not in dist):
                dist[nxt] = dist[u] + 1
                queue.append(nxt)
                predecessors[nxt] = predecessors[u].copy()
                predecessors[nxt].add(u)
                
def backward_bfs(G, v, D, S, V, s=None, d=None):
    dist = {v: 0}
    predecessors = {v: set()}
    if (s is None):
        s = v
    else:
        predecessors[s] = set([v])
        dist[s] = d
    queue = [s]
    while (queue):
        u = queue.pop(0)
        if (u in S):
            D.add_edge(u, v, weight=dist[u])
        elif (not ((predecessors[u] - set([v])) & S)):
            V.add(u)
            D.add_edge(u, v, weight=dist[u])
        if (dist[u] >= k):
            continue
        for nxt in G.predecessors(u):
            if (nxt not in dist):
                dist[nxt] = dist[u] + 1
                queue.append(nxt)
                predecessors[nxt] = predecessors[u].copy()
                predecessors[nxt].add(u)

def construct_d(G):
    temp = G.copy()
    S = set()
    D = nx.DiGraph()
    V = set()
    
    while (len(S) <= b and temp.number_of_nodes() > 0):
        v = largest_degree_node(temp)
        # Process graph
        forward_bfs(G, v, D, S, V)
        # Process reverse graph
        backward_bfs(G, v, D, S, V)
        
        S.add(v)
        temp.remove_nodes_from(V)
        temp.remove_nodes_from(nx.isolates(temp))
        
    return (D, V, S)

In [29]:
D1, VD1, S = construct_d(G)

In [30]:
print("n: {}, m: {}, |S|: {}".format(D1.number_of_nodes(), D1.number_of_edges(), len(S)))

n: 12606, m: 22417, |S|: 131


Construction of $D_2$ at the second level

In [31]:
b = 10000

In [32]:
Gp = G.copy()
Gp.remove_nodes_from(S)

In [33]:
print("n: {}, m: {}".format(Gp.number_of_nodes(), Gp.number_of_edges()))

n: 12553, m: 1349


In [34]:
D2, VD2, Sp = construct_d(Gp)

In [35]:
print("n: {}, m: {}, |S'|: {}".format(D2.number_of_nodes(), D2.number_of_edges(), len(Sp)))

n: 981, m: 1030, |S'|: 68


Tuning?

In [ ]:
VD1capSp = VD1 & Sp

In [ ]:
# for u in VD1capSp:
#     for v in VD1capSp:
#         if ()

Construction of residual graph $D_3$

In [36]:
D3 = Gp.copy()
D3.remove_nodes_from(Sp)

In [37]:
print("n: {}, m: {}".format(D3.number_of_nodes(), D3.number_of_edges()))

n: 12485, m: 566


Querying

In [16]:
def single_intermediate(s, t, D, S):
    if (s not in D.nodes() or t not in D.nodes()):
        return False
    cand = set(D.successors(s)) & set(D.predecessors(t)) & S
    for v in cand:
        if (D.edge[s][v]['weight'] + D.edge[v][t]['weight'] <= k):
            return True
    return False

def double_intermediate(s, t, D, S):
    if (s not in D.nodes() or t not in D.nodes()):
        return False
    candu = set(D.successors(s)) & S
    candv = set(D.predecessors(t)) & S
    for u in candu:
        for v in candv:
            if (D.has_edge(u, v)):
                if (D.edge[s][u]['weight'] + D.edge[u][v]['weight'] + D.edge[v][t]['weight'] <= k):
                    return True
    return False

cases = {case: 0 for case in range(1,6)}

def query(s, t):
    # Case 1: both s and t are in S
    if (s in S and t in S):
        cases[1] = cases[1] + 1
        if (D1.has_edge(s, t)):
            return True
        else:
            return False
    # Case 2: s or t is in S, but not both
    elif (s in S or t in S):
        cases[2] = cases[2] + 1
        if (D1.has_edge(s, t) or single_intermediate(s, t, D1, S)):
            return True
        else:
            return False
    # Case 3: both s and t are in S'
    elif (s in Sp and t in Sp):
        cases[3] = cases[3] + 1
        if (D2.has_edge(s, t) or double_intermediate(s, t, D1, S)):
            return True
        else:
            return False
    # Case 4: s or t is in S'
    elif (s in Sp or t in Sp):
        cases[4] = cases[4] + 1
        if (D2.has_edge(s, t) or single_intermediate(s, t, D2, Sp) or double_intermediate(s, t, D1, S)):
            return True
        else:
            return False
    # Case 5: both s and t are not in S or S'
    elif (s in D3.nodes() and t in D3.nodes()):
        cases[5] = cases[5] + 1
        if (double_intermediate(s, t, D1, S) or double_intermediate(s, t, D2, Sp)):
            return True
        else:
            try:
                return nx.shortest_path_length(D3, s, t) <= k
            except:
                return False

Test correctness

In [47]:
dist_test = []
for i in range(10000):
    s = rnd.choice(G.nodes())
    t = rnd.choice(G.nodes())
    d = 0    
    try:
        d = nx.shortest_path_length(G, s, t)
    except:
        pass
    dist_test.append((s, t, d))

In [48]:
# No output - good!
for s, t, d in dist_test:
    q = query(s, t)
    if (not q and d <= 0):
        continue
    if (q ^ (d <= k)):
        print("Error in pair ({}, {})".format(s, t))

Edge insertion

In [19]:
def update_graph(G, s, t):
    if (s in G.nodes() and t in G.nodes()):
        G.add_edge(s, t)

In [40]:
print("({}, {})".format(rnd.choice(G.nodes()), rnd.choice(G.nodes())))

(6899, 9534)


In [41]:
G.add_edge(6899, 9534)
update_graph(Gp, 6899, 9534)
update_graph(D3, 6899, 9534)

In [42]:
print("n: {}, m: {}".format(G.number_of_nodes(), G.number_of_edges()))
print("n: {}, m: {}".format(Gp.number_of_nodes(), Gp.number_of_edges()))
print("n: {}, m: {}".format(D3.number_of_nodes(), D3.number_of_edges()))

n: 12684, m: 13409
n: 12553, m: 1350
n: 12485, m: 567


In [22]:
def update_d(G, s, t, D, S, V):
    # Process graph
    for v in D.predecessors(s): # if s is reachable from v
        if (D.edge[v][s]['weight'] < k): # if we can go further than s
            if (D.has_edge(v, t)):
                if (D.edge[v][t]['weight'] > D.edge[v][s]['weight'] + 1):
                    D.edge[v][t]['weight'] = D.edge[v][s]['weight'] + 1
                    forward_bfs(G, v, D, S, V, t, D.edge[v][t]['weight'])
            else:
                D.add_edge(v, t, weight=D.edge[v][s]['weight'] + 1)
                forward_bfs(G, v, D, S, V, t, D.edge[v][t]['weight'])
    # Process reverse graph
    for v in D.successors(t):
        if (D.edge[t][v]['weight'] < k):
            if (D.has_edge(s, v)):
                if (D.edge[s][v]['weight'] > D.edge[t][v]['weight'] + 1):
                    D.edge[s][v]['weight'] = D.edge[t][v]['weight'] + 1
                    backward_bfs(G, v, D, S, V, s, D.edge[s][v]['weight'])
            else:
                D.add_edge(s, v, weight=D.edge[t][v]['weight'] + 1)
                backward_bfs(G, v, D, S, V, s, D.edge[s][v]['weight'])

In [43]:
update_d(G, 2, 7, D1, S, VD1)
update_d(Gp, 2, 7, D2, Sp, VD2)

In [44]:
print("n: {}, m: {}, |S|: {}".format(D1.number_of_nodes(), D1.number_of_edges(), len(S)))
print("n: {}, m: {}, |S'|: {}".format(D2.number_of_nodes(), D2.number_of_edges(), len(Sp)))

n: 12606, m: 22417, |S|: 131
n: 982, m: 1031, |S'|: 68
